In [58]:
import os
import pandas as pd
import matplotlib
from sklearn.preprocessing import LabelEncoder


#be sure to fix your own path for DATA_PATH
DATA_PATH = "C:/Programming/DoomPatrol/Kaggle_2017_Titanic/"
def load_train_data(data_path = DATA_PATH):
    csv_path = os.path.join(data_path, "train.csv")
    return pd.read_csv(csv_path)

def load_test_data(data_path = DATA_PATH):
    csv_path = os.path.join(data_path, "test.csv")
    return pd.read_csv(csv_path)

def parseOutAnswers(dataFrame):
    answers = {}

    for passengerIndex, passenger in dataFrame.iterrows():
        passengerID = passenger['PassengerId']
        answers[passengerID] = passenger['Survived']

    return answers

def cleanData(df):
    answers = df['Survived']
    columns = ['PassengerId', 'Survived', 'Name', 'Sex', 'Ticket', 'Cabin','Embarked', 'SibSp']
    df.drop(columns, axis=1, inplace=True)
    median = df['Age'].median()
    df['Age'].fillna(median, inplace=True)
    return df
    
def scoreClassifier(answerDict, predictionDict):
    score = 0
    for key in answerDict.keys():
        if answerDict[key] == predictionDict[key]:
            score += 1

    return float(score) / len(answerDict)

In [59]:
train_data = load_train_data()
train_data_copy = train_data.copy()


#pclass = Ticket Class (first class, second class, third class)
#sibsp = Number of siblings and spouses aboard
#parch = Number of parents and children aboard
#ticket = Ticket number
#cabin = cabin number
#embarked = Port of embarkation (C = Cherbourg, Q = Queenstown, S = Southampton)

###### Run this commented out code to get data specifics ##########3
#import matplotlib.pyplot as plt
#train_data_copy.head()
#train_data_copy.info()
#train_data_copy.describe()
#train_data_copy.hist(bins = 30, figsize = (15,15))
#plt.show()

In [60]:
#this block encodes the sex data as binary
#and prints the correlations
# 0 = female 1 = male

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
train_data_copy["Sex Code"] = encoder.fit_transform(train_data_copy["Sex"])
corr_matrix = train_data_copy.corr()
corr_matrix["Survived"].sort_values(ascending=False)

Survived       1.000000
Fare           0.257307
Parch          0.081629
PassengerId   -0.005007
SibSp         -0.035322
Age           -0.077221
Pclass        -0.338481
Sex Code      -0.543351
Name: Survived, dtype: float64

In [61]:
train_data_copy.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Sex Code
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208,0.647587
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429,0.477990
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400,0.000000
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200,1.000000
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000,1.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200,1.000000


In [62]:

from sklearn import datasets
from sklearn.naive_bayes import GaussianNB

answers = train_data_copy['Survived']
train_clean = train_data_copy.copy()
columns = ['PassengerId', 'Survived', 'Name', 'Sex', 'Ticket', 'Cabin','Embarked', 'SibSp']
train_clean.drop(columns, axis=1, inplace=True)
median = train_clean['Age'].median()
train_clean['Age'].fillna(median, inplace=True)
model = GaussianNB()
model.fit(train_clean, answers)
train_clean.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 5 columns):
Pclass      891 non-null int64
Age         891 non-null float64
Parch       891 non-null int64
Fare        891 non-null float64
Sex Code    891 non-null int64
dtypes: float64(2), int64(3)
memory usage: 34.9 KB


In [63]:
from sklearn.preprocessing import LabelEncoder


test= load_test_data()
answers = parseOutAnswers(test)
encoder = LabelEncoder()
test["Sex Code"] = encoder.fit_transform(test["Sex"])
columns = ['PassengerId', 'Name', 'Sex', 'Ticket', 'Cabin','Embarked', 'SibSp']
test.drop(columns, axis=1, inplace=True)
median = test['Age'].median()
test['Age'].fillna(median, inplace=True)
median = test['Fare'].median()
test['Fare'].fillna(median, inplace=True)


print("Predictions:\t", model.predict(test))

KeyError: 'Survived'

In [64]:
print(scoreClassifier(answers, model.predict(test)))

ValueError: could not convert string to float: 'C'